In [ ]:
file=r'C:\Users\Admin\Downloads\p3DL\P3 Quora Text Classification with embeddings\train.csv'

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
pdl=pd.read_csv(file)

In [ ]:
pdl.head()

In [ ]:
pdl.tail()

In [ ]:
embedding_index={}

f=open(r'C:\Users\Admin\Downloads\p3DL\glove.840B.300d\glove.840B.300d.txt',encoding='utf-8')


for line in f:
    values=line.split()
    word=values[0]

    wts=np.asarray(values[1:],dtype='float32')

    embedding_index[word]=wts

f.close()


In [ ]:
print('Found %s word vectors.' % len(embeddings_index))

In [ ]:
from sklearn.model_selection import train_test_split
pdl_train,pdl_test=train_test_split(pdl,test_size=0.2)

In [ ]:
x_train=pdl_train["question_text"]
y_train=pdl_train["target"]
x_test=pdl_test["question_text"]
y_test=pdl_test["target"]

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.layers import LSTM, Dense, Input , Embedding,Dropout
from tensorflow.keras.models import Model

In [ ]:
max_len=40

tk=Tokenizer(char_level=False,split=' ')

tk.fit_on_texts(x_train)


In [ ]:
vocab_size=len(tk.word_index)
vocab_size

In [ ]:
seq_train=tk.texts_to_sequences(x_train)
seq_test=tk.texts_to_matrix(x_test)

In [ ]:
seq_train_matrix=sequence.pad_sequences(seq_train,maxlen=max_len)
seq_test_matrix=sequence.pad_sequences(seq_test,maxlen=max_len)

In [ ]:
embedding_matrix=np.zeros((vocab_size+1,100))

In [ ]:
for word,i in tk.word_index.items():
    embed_vector=embedding_index.get(word)

    if embed_vector is not None:
        embedding_matrix[i]=embed_vector
    if embed_vector is None:
        print(word)

In [ ]:
inputs=Input(name='review',shape=[max_len])
embed=Embedding(vocab_size+1,100,input_length=max_len,mask_zero=True,
                weights=[embedding_matrix],trainable=False)(inputs)

lstm_layer=LSTM(100)(embed)

dense1=Dense(10,activation='relu')(lstm_layer)
dr1=Dropout(0.2)(dense1)

final_layer=Dense(1,activation='sigmoid')(dr1)

model=Model(inputs=inputs,outputs=final_layer)

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit(seq_train_matrix,y_train,validation_data=[seq_test_matrix,y_test],
          epochs=10,batch_size=1000)